# Breast Cancer Prediction Using a Custom Neural Network (No External Libraries Outside of Python)

In [1]:
# Importing Required Python Libraries  
import math
import random

In [2]:
# Define Matrix Multiplication (for Weighted Sum Calculation in Neural Networks)
def matrix_multiplication(matrix1, matrix2):
    if len(matrix1[0]) != len(matrix2):
        raise ValueError('Matrix 1 columns do not match with Matrix 2 rows for multiplication')
    result_matrix = [[0 for _ in range(len(matrix2[0]))] for _ in range(len(matrix1))]
    
    for i in range(len(matrix1)):
        for j in range(len(matrix2[0])):
            for k in range(len(matrix2)):
                result_matrix[i][j] += matrix1[i][k] * matrix2[k][j]   
    return result_matrix

In [3]:
# Define Artificial Neural Network Class with Essential Methods
class ArtificialNeuralNetwork:
    def __init__(self, learning_rate=0.0001):
        self.learning_rate = learning_rate
        self.layers = []
        self.loss_function = "mse"

    def add_input_layers(self, input_dimension): 
        self.input_dimension = input_dimension

    def initialize_weights(self, input_dim, output_dim):
        #std_dev = math.sqrt(2.0 / input_dim)
        std_dev = math.sqrt(1.0 / input_dim)  # Xavier 
        weights = [[random.gauss(0, std_dev) for _ in range(output_dim)] for _ in range(input_dim)]
        biases = [[0 for _ in range(output_dim)]]
        return weights, biases

    def add_hidden_layer(self, number_of_neurons, activation):
        input_dim = self.input_dimension if not self.layers else len(self.layers[-1][0][0])
        weights, biases = self.initialize_weights(input_dim, number_of_neurons)
        self.layers.append((weights, biases, activation))
        #print('\nself.layers:\n',self.layers)

    def add_activation_function(self, x, activation):
        if activation == 'relu':
            return [[max(0, val) for val in row] for row in x]
        elif activation == 'sigmoid':
            return [[1 / (1 + math.exp(-val)) for val in row] for row in x]
        elif activation == 'linear':
            return x
        else:
            raise ValueError("Invalid activation function.")
 
    def forward_propagation(self, input_data):
        activations = [input_data]
        pre_activations = []  # Store Z values (pre-activation)
        for weights, biases, activation in self.layers:
            z = matrix_multiplication(activations[-1], weights)
            for i in range(len(z)):
                for j in range(len(z[0])):
                    z[i][j] += biases[0][j]
            pre_activations.append(z)  # Store pre-activation values
            a = self.add_activation_function(z, activation)
            activations.append(a)
        #print('\nactivations:\n',activations)
        return activations, pre_activations 
    
    def predict(self, X):
        activations, pre_activations=self.forward_propagation(X)
        return activations[-1]
    
    def compiles(self, loss="mse"):
        if loss not in ["mse", "binary_crossentropy"]:
            raise ValueError("Loss function must be 'mse' or 'binary_crossentropy'")
        self.loss_function = loss
    
        
    def compute_loss(self, y_true, y_pred):
        if isinstance(y_true[0], int) or isinstance(y_true[0], float):  
            y_true = [[val] for val in y_true]  # Convert to nested list
        
        if isinstance(y_pred[0], float):  
            y_pred = [[val] for val in y_pred]  # Convert to nested list
            
        if self.loss_function == "mse":
            return sum(
            [(y_true[i][0] - y_pred[i][0])**2 for i in range(len(y_true))]
            ) / len(y_true)
        
        elif self.loss_function == "binary_crossentropy":
            return -sum([
            y_true[i][0] * math.log(max(y_pred[i][0], 1e-15)) + 
            (1 - y_true[i][0]) * math.log(max(1 - y_pred[i][0], 1e-15))
            for i in range(len(y_true))
            ]) / len(y_true)
    
    def derivative_activation(self, x, activation):
        if activation == 'relu':
            return 1 if x > 0 else 0
        elif activation == 'sigmoid':
            sigmoid = 1 / (1 + math.exp(-x))
            return sigmoid * (1 - sigmoid)
        elif activation == 'linear':
            return 1
        else:
            raise ValueError("Invalid activation function.")
    
    def backward_propagation(self, activations,pre_activations, y_true):
        m = len(y_true)
        y_pred = activations[-1]

        if self.loss_function=="mse":
            dA = [[(y_pred[i][j] - y_true[i][j]) for j in range(len(y_true[0]))] for i in range(len(y_true))]
        elif self.loss_function=="binary_crossentropy":
            #dA = [[(y_pred[i][j] - y_true[i][j]) / (y_pred[i][j] * (1 - y_pred[i][j]))for j in range(len(y_true[0]))] for i in range(len(y_true))]    
            epsilon = 1e-12
            dA = [[(y_pred[i][j] - y_true[i][j]) / (y_pred[i][j] * (1 - y_pred[i][j]) + epsilon) for j in range(len(y_true[0]))] for i in range(len(y_true))]

        for i in reversed(range(len(self.layers))):  
            weights, biases, activation = self.layers[i]
            A_prev = activations[i]
            Z_current = pre_activations[i]  

            dZ = [[dA[j][k] * self.derivative_activation(Z_current[j][k], activation) for k in range(len(dA[0]))] for j in range(len(dA))]
    
            dW = [[sum(A_prev[j][k] * dZ[j][l] for j in range(m)) / m for l in range(len(dZ[0]))] for k in range(len(A_prev[0]))]
            
            db = [[sum(dZ[j][k] for j in range(m)) / m for k in range(len(dZ[0]))]]

            for j in range(len(weights)):
                for k in range(len(weights[j])):
                    weights[j][k] -= self.learning_rate * dW[j][k]
            for j in range(len(biases[0])):
                biases[0][j] -= self.learning_rate * db[0][j]

            dA = matrix_multiplication(dZ, [list(row) for row in zip(*weights)])  # Using transposed weights

            self.layers[i] = (weights, biases, activation)

    def fit(self, X, y, epochs=1000, batch_size=32):
        print('Training Loss:')
        print('='*25)
        num_samples = len(X)
        for epoch in range(epochs):
            indices = list(range(num_samples))
            random.shuffle(indices)  
            X = [X[i] for i in indices]
            y = [y[i] for i in indices]
            
            for start in range(0, num_samples, batch_size):
                end = start + batch_size
                batch_X = X[start:end]  # Extract batch input
                batch_y = y[start:end]  # Extract batch output
                
                activations, pre_activations = self.forward_propagation(batch_X)
                loss = self.compute_loss(batch_y, activations[-1])
                self.backward_propagation(activations, pre_activations, batch_y)
            
            if epoch % 200 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}") 

In [4]:
# Define Customized Label Encoder Function 
def label_encoder(data, target_index):
    target_values = set(row[target_index] for row in data)
    label_mapping = {category: index for index, category in enumerate(target_values)}
    
    # Apply the label encoding
    for row in data:
        row[target_index] = label_mapping[row[target_index]]
    
    return data, label_mapping

In [5]:
# Define Customized Standardization Function 
def mean(data):
    return [sum(col) / len(col) for col in zip(*data)]

# Function to calculate the standard deviation of each column
def std_dev(data, means):
    return [((sum((x_i - mu)**2 for x_i in col) / len(col))**0.5) for col, mu in zip(zip(*data), means)]

# Standardizing function
def standardize(data):
    means = mean(data)
    std_devs = std_dev(data, means)
    standardized_data = []
    for row in data:
        standardized_row = []
        for x_i, mu, sigma in zip(row, means, std_devs):
           
            if sigma == 0: # Prevent division by zero: If std_dev is zero, set the value to zero
                standardized_row.append(0)
            else:
                standardized_row.append((x_i - mu) / sigma)
        standardized_data.append(standardized_row)
    return standardized_data

In [6]:
# Define Training and Testing Dataset Splitting Function  
def train_test_spliting(X,y,test_size=0.2):
    indices=list(range(len(X)))
    random.shuffle(indices)
    X_shuffle=[X[i] for i in indices]
    y_shuffle=[y[i] for i in indices]
    
    split_index = int((1 - test_size) * len(X)) 
    
    X_train=X_shuffle[:split_index]
    y_train=y_shuffle[:split_index]
    X_test=X_shuffle[split_index:]
    y_test=y_shuffle[split_index:]

    return X_train, y_train, X_test, y_test

In [7]:
# Customized Metrics for Evaluating Classification Performance 
# Accuracy function
def accuracy(y_true, y_pred):
    correct_predictions = sum([1 for true, pred in zip(y_true, y_pred) if true == pred])
    return correct_predictions / len(y_true)

# Precision function
def precision(y_true, y_pred):
    tp = sum([1 for true, pred in zip(y_true, y_pred) if true == pred == 1])
    fp = sum([1 for true, pred in zip(y_true, y_pred) if true == 0 and pred == 1])
    
    # Avoid division by zero
    if tp + fp == 0:
        return 0.0
    return tp / (tp + fp)

# Recall function
def recall(y_true, y_pred):
    tp = sum([1 for true, pred in zip(y_true, y_pred) if true == pred == 1])
    fn = sum([1 for true, pred in zip(y_true, y_pred) if true == 1 and pred == 0])
    
    # Avoid division by zero
    if tp + fn == 0:
        return 0.0
    return tp / (tp + fn)

# F-Score function
def f_score(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    
    # Avoid division by zero
    if (prec + rec) == 0:
        return 0.0
    return 2 * (prec * rec) / (prec + rec)

# Custom classification report function
def classification_report(y_true, y_pred):
    # Calculate metrics
    accuracy_val = accuracy(y_true, y_pred)
    precision_val = precision(y_true, y_pred)
    recall_val = recall(y_true, y_pred)
    f_score_val = f_score(y_true, y_pred)
    
    # Create a report dictionary to display metrics
    report = {
        'Accuracy': accuracy_val,'Precision': precision_val,'Recall': recall_val,'F-Score': f_score_val
    }
   
    # Printing the results in the desired format
    print('\nClassification Report:')
    print(f"{'Accuracy':<10} | {'Precision':<10} | {'Recall':<10} | {'F-Score':<10}")
    print('-' * 35)  # This creates a separator line
    print(f"{accuracy_val:<10.4f} | {precision_val:<10.4f} | {recall_val:<10.4f} | {f_score_val:<10.4f}")

In [8]:
# Load Dataset-Breast Cancer Wisconsin (Diagnostic)

# Dataset download Link 1:https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic
# Dataset download Link 2:https://github.com/pkmklong/Breast-Cancer-Wisconsin-Diagnostic-DataSet?tab=readme-ov-file

import csv
with open('BreastCancerData.csv', newline='') as salimcsvfile2:
    datareader = csv.reader(salimcsvfile2, delimiter=',', quotechar='"')
    
    # Convert numeric values to float, keep strings unchanged
    data = []
    for row in datareader:
        converted_row = [float(value) if value.replace('.', '', 1).isdigit() else value for value in row]
        data.append(converted_row)

display('raw data:', data) # Raw iris data 
new_data=data[1:100] # select only 2 categories data out of 3 
display('new_data_two_cat',new_data)

'raw data:'

[['id',
  'diagnosis',
  'radius_mean',
  'texture_mean',
  'perimeter_mean',
  'area_mean',
  'smoothness_mean',
  'compactness_mean',
  'concavity_mean',
  'concave points_mean',
  'symmetry_mean',
  'fractal_dimension_mean',
  'radius_se',
  'texture_se',
  'perimeter_se',
  'area_se',
  'smoothness_se',
  'compactness_se',
  'concavity_se',
  'concave points_se',
  'symmetry_se',
  'fractal_dimension_se',
  'radius_worst',
  'texture_worst',
  'perimeter_worst',
  'area_worst',
  'smoothness_worst',
  'compactness_worst',
  'concavity_worst',
  'concave points_worst',
  'symmetry_worst',
  'fractal_dimension_worst',
  ''],
 [842302.0,
  'M',
  17.99,
  10.38,
  122.8,
  1001.0,
  0.1184,
  0.2776,
  0.3001,
  0.1471,
  0.2419,
  0.07871,
  1.095,
  0.9053,
  8.589,
  153.4,
  0.006399,
  0.04904,
  0.05373,
  0.01587,
  0.03003,
  0.006193,
  25.38,
  17.33,
  184.6,
  2019.0,
  0.1622,
  0.6656,
  0.7119,
  0.2654,
  0.4601,
  0.1189],
 [842517.0,
  'M',
  20.57,
  17.77,
  132.9,

'new_data_two_cat'

[[842302.0,
  'M',
  17.99,
  10.38,
  122.8,
  1001.0,
  0.1184,
  0.2776,
  0.3001,
  0.1471,
  0.2419,
  0.07871,
  1.095,
  0.9053,
  8.589,
  153.4,
  0.006399,
  0.04904,
  0.05373,
  0.01587,
  0.03003,
  0.006193,
  25.38,
  17.33,
  184.6,
  2019.0,
  0.1622,
  0.6656,
  0.7119,
  0.2654,
  0.4601,
  0.1189],
 [842517.0,
  'M',
  20.57,
  17.77,
  132.9,
  1326.0,
  0.08474,
  0.07864,
  0.0869,
  0.07017,
  0.1812,
  0.05667,
  0.5435,
  0.7339,
  3.398,
  74.08,
  0.005225,
  0.01308,
  0.0186,
  0.0134,
  0.01389,
  0.003532,
  24.99,
  23.41,
  158.8,
  1956.0,
  0.1238,
  0.1866,
  0.2416,
  0.186,
  0.275,
  0.08902],
 [84300903.0,
  'M',
  19.69,
  21.25,
  130.0,
  1203.0,
  0.1096,
  0.1599,
  0.1974,
  0.1279,
  0.2069,
  0.05999,
  0.7456,
  0.7869,
  4.585,
  94.03,
  0.00615,
  0.04006,
  0.03832,
  0.02058,
  0.0225,
  0.004571,
  23.57,
  25.53,
  152.5,
  1709.0,
  0.1444,
  0.4245,
  0.4504,
  0.243,
  0.3613,
  0.08758],
 [84348301.0,
  'M',
  11.42,
  20.38,

In [9]:
# Apply the custom label encoder to the target variable (last column)
new_data, label_mapping = label_encoder(new_data, target_index=1)
# Display the modified data and label mapping
display('Labeld Data:',new_data)
display('Label Mapping:', label_mapping)

'Labeld Data:'

[[842302.0,
  0,
  17.99,
  10.38,
  122.8,
  1001.0,
  0.1184,
  0.2776,
  0.3001,
  0.1471,
  0.2419,
  0.07871,
  1.095,
  0.9053,
  8.589,
  153.4,
  0.006399,
  0.04904,
  0.05373,
  0.01587,
  0.03003,
  0.006193,
  25.38,
  17.33,
  184.6,
  2019.0,
  0.1622,
  0.6656,
  0.7119,
  0.2654,
  0.4601,
  0.1189],
 [842517.0,
  0,
  20.57,
  17.77,
  132.9,
  1326.0,
  0.08474,
  0.07864,
  0.0869,
  0.07017,
  0.1812,
  0.05667,
  0.5435,
  0.7339,
  3.398,
  74.08,
  0.005225,
  0.01308,
  0.0186,
  0.0134,
  0.01389,
  0.003532,
  24.99,
  23.41,
  158.8,
  1956.0,
  0.1238,
  0.1866,
  0.2416,
  0.186,
  0.275,
  0.08902],
 [84300903.0,
  0,
  19.69,
  21.25,
  130.0,
  1203.0,
  0.1096,
  0.1599,
  0.1974,
  0.1279,
  0.2069,
  0.05999,
  0.7456,
  0.7869,
  4.585,
  94.03,
  0.00615,
  0.04006,
  0.03832,
  0.02058,
  0.0225,
  0.004571,
  23.57,
  25.53,
  152.5,
  1709.0,
  0.1444,
  0.4245,
  0.4504,
  0.243,
  0.3613,
  0.08758],
 [84348301.0,
  0,
  11.42,
  20.38,
  77.58

'Label Mapping:'

{'M': 0, 'B': 1}

In [10]:
# Separate Input Features and Target Labels
num_of_col=len(new_data[0])
X= [row[2:num_of_col] for row in new_data]
y=[[row[1]] for row in new_data]
display('X:',X)
display('y:',y)

'X:'

[[17.99,
  10.38,
  122.8,
  1001.0,
  0.1184,
  0.2776,
  0.3001,
  0.1471,
  0.2419,
  0.07871,
  1.095,
  0.9053,
  8.589,
  153.4,
  0.006399,
  0.04904,
  0.05373,
  0.01587,
  0.03003,
  0.006193,
  25.38,
  17.33,
  184.6,
  2019.0,
  0.1622,
  0.6656,
  0.7119,
  0.2654,
  0.4601,
  0.1189],
 [20.57,
  17.77,
  132.9,
  1326.0,
  0.08474,
  0.07864,
  0.0869,
  0.07017,
  0.1812,
  0.05667,
  0.5435,
  0.7339,
  3.398,
  74.08,
  0.005225,
  0.01308,
  0.0186,
  0.0134,
  0.01389,
  0.003532,
  24.99,
  23.41,
  158.8,
  1956.0,
  0.1238,
  0.1866,
  0.2416,
  0.186,
  0.275,
  0.08902],
 [19.69,
  21.25,
  130.0,
  1203.0,
  0.1096,
  0.1599,
  0.1974,
  0.1279,
  0.2069,
  0.05999,
  0.7456,
  0.7869,
  4.585,
  94.03,
  0.00615,
  0.04006,
  0.03832,
  0.02058,
  0.0225,
  0.004571,
  23.57,
  25.53,
  152.5,
  1709.0,
  0.1444,
  0.4245,
  0.4504,
  0.243,
  0.3613,
  0.08758],
 [11.42,
  20.38,
  77.58,
  386.1,
  0.1425,
  0.2839,
  0.2414,
  0.1052,
  0.2597,
  0.09744,


'y:'

[[0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [1],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [0],
 [0],
 [1],
 [0],
 [1],
 [0],
 [0],
 [1],
 [1],
 [1],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [0],
 [0],
 [1],
 [1],
 [1]]

In [11]:
# Data Standardization
X= standardize(X)
print('x_standardized:',X)

x_standardized: [[0.979156878939792, -2.47698790601615, 1.1346505397540376, 0.9281444882568615, 1.2412561923936043, 2.469299211285299, 2.348452841990355, 2.184917286828481, 1.5827994922697357, 1.7179543351907918, 2.589853901703916, -0.5298254734595328, 2.7021885670656554, 3.104225561463173, -0.21920822470043247, 0.9807482937308922, 0.5536527962374539, 0.4726140670228787, 0.793159348964196, 0.7256951771188435, 1.6389328777520253, -1.6229855930139474, 2.01572270445372, 1.8477215644988862, 0.8683546891887729, 1.6329947114304193, 1.4974228355134747, 1.7579355980693618, 1.6905645882319202, 1.1230199013390194], [1.749508671070447, -0.5111274741355691, 1.5702469839519306, 1.9434731243216425, -1.3198016318250938, -0.7864796683845419, -0.35597822895865044, 0.1657379452613507, -0.38686648182356176, -0.9803667425610889, 0.29658717721156025, -0.8627759933781107, 0.022835880992549747, 0.7770661365862443, -0.7048676818874801, -0.799331227605176, -0.49975668097845954, 0.047596165005680965, -0.7590276

In [12]:
# Split the Dataset into Training and Testing Sets 
X_train, y_train, X_test, y_test=train_test_spliting(X,y,test_size=0.2)
display('X_train:',X_train)
display('y_train:',y_train)
display('X_test:', X_test)
display('y_test:', y_test)

'X_train:'

[[0.08339898111344998,
  1.4653736096090475,
  -0.041028575972155215,
  -0.015954938877836614,
  -0.6251356206808066,
  -1.2337074366865963,
  -1.1541152035557962,
  -0.9151122469358988,
  -1.1883615322371441,
  -1.1799249710881108,
  3.084683893026836,
  1.9618643275880747,
  2.4379179881272175,
  1.7135629701978374,
  -0.0189874604086685,
  -0.9052647697536516,
  -0.5123508164874456,
  1.0404517498798642,
  -1.3368182294561324,
  -0.8740456268370552,
  -0.5396928249603109,
  -0.23867017133707336,
  -0.6274732737264315,
  -0.5715676754405606,
  -2.1103239084779313,
  -1.3660582461315478,
  -1.4055201437894875,
  -1.705673475791891,
  -2.0835404290544925,
  -1.558602029025142],
 [-0.7018821093143106,
  0.56091820116602,
  -0.7207315423641145,
  -0.7429302423002196,
  -1.0930650805425106,
  -0.5277650318365915,
  -0.6953006077010797,
  -0.6930628873927048,
  -0.003966012395210574,
  -0.07806972155240643,
  -0.7250912166375274,
  0.6292852338649446,
  -0.593967521021968,
  -0.78170819830

'y_train:'

[[0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [0],
 [0],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [1],
 [1],
 [0],
 [0],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [1],
 [0],
 [1],
 [0],
 [0],
 [1],
 [1],
 [0],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [1],
 [0],
 [0]]

'X_test:'

[[-0.31969207290840407,
  -0.8915307782748425,
  -0.40028782153338793,
  -0.41271412897084336,
  -1.9201208900866953,
  -1.0818496361069012,
  -1.222740776228564,
  -1.2237766048824665,
  -1.8762844095151983,
  -0.6302216299308538,
  -1.184992032102359,
  -0.49466558775051145,
  -0.9831472407867765,
  -0.9694769870007897,
  -1.0142997721565696,
  -0.8641783959297101,
  -0.7339476293717496,
  -0.8891639396832367,
  -0.8205765985657439,
  -0.5868202560884966,
  -0.6067919803855224,
  -0.5922246818288794,
  -0.6114467092955168,
  -0.6487166505969388,
  -1.4551279576081733,
  -0.8442057487218575,
  -1.0636250975877113,
  -0.8724792453004168,
  -1.1126655876511038,
  -0.49997865188012947],
 [-0.6750093723795203,
  -1.0617812080994127,
  -0.6004034156995485,
  -0.7085652730795655,
  1.9564654481171853,
  0.7085341959431455,
  0.5433846365680124,
  0.4471054552262493,
  0.5054863070984427,
  1.4020891636572235,
  -0.5724839840194503,
  -0.5591577538024181,
  -0.5867413723791199,
  -0.59863362

'y_test:'

[[1],
 [0],
 [1],
 [1],
 [1],
 [0],
 [1],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [0],
 [1],
 [0],
 [0]]

In [13]:
# Initialize, Train Custom Artificial Neural Network (ANN)  

# Create an Object for the Custom ANN with Initialization 
my_ANN = ArtificialNeuralNetwork(learning_rate=0.0001)

# Form a Sequential Feedforward ANN (for Binary Classification)
my_ANN.add_input_layers(input_dimension=len(X_train[0])) 
my_ANN.add_hidden_layer(number_of_neurons=15, activation='relu')
my_ANN.add_hidden_layer(number_of_neurons=20, activation='relu')
my_ANN.add_hidden_layer(number_of_neurons=1, activation='sigmoid')

# Compile the Model with a Loss Function
my_ANN.compiles(loss="binary_crossentropy")

# Train the Model Using Mini-Batch Gradient Descent
my_ANN.fit(X_train, y_train, epochs=10000, batch_size=32)  

Training Loss:
Epoch 0, Loss: 0.6708
Epoch 200, Loss: 0.6342
Epoch 400, Loss: 0.6879
Epoch 600, Loss: 0.6432
Epoch 800, Loss: 0.6145
Epoch 1000, Loss: 0.6017
Epoch 1200, Loss: 0.5528
Epoch 1400, Loss: 0.5397
Epoch 1600, Loss: 0.5724
Epoch 1800, Loss: 0.5681
Epoch 2000, Loss: 0.5412
Epoch 2200, Loss: 0.4966
Epoch 2400, Loss: 0.4539
Epoch 2600, Loss: 0.4769
Epoch 2800, Loss: 0.4499
Epoch 3000, Loss: 0.4356
Epoch 3200, Loss: 0.3868
Epoch 3400, Loss: 0.3677
Epoch 3600, Loss: 0.4540
Epoch 3800, Loss: 0.4047
Epoch 4000, Loss: 0.3677
Epoch 4200, Loss: 0.3484
Epoch 4400, Loss: 0.2861
Epoch 4600, Loss: 0.3576
Epoch 4800, Loss: 0.3743
Epoch 5000, Loss: 0.3613
Epoch 5200, Loss: 0.2953
Epoch 5400, Loss: 0.3116
Epoch 5600, Loss: 0.2936
Epoch 5800, Loss: 0.2915
Epoch 6000, Loss: 0.3007
Epoch 6200, Loss: 0.2669
Epoch 6400, Loss: 0.2503
Epoch 6600, Loss: 0.2496
Epoch 6800, Loss: 0.2469
Epoch 7000, Loss: 0.1867
Epoch 7200, Loss: 0.2598
Epoch 7400, Loss: 0.2738
Epoch 7600, Loss: 0.2084
Epoch 7800, Loss:

In [14]:
# Test Custom Artificial Neural Network (ANN) on New Data
predicted_output = my_ANN.predict(X_test)
#print("Prediction:", predicted_output)
y_test_predicted = [[1 if val > 0.50 else 0 for val in row] for row in predicted_output]   
print("y_test_predicted:", y_test_predicted)

# Flatten the y_test and y_test_predicted lists 
y_test = [item[0] for item in y_test]
y_test_predicted = [item[0] for item in y_test_predicted]
# Call custom classification report
classification_report(y_test, y_test_predicted)

y_test_predicted: [[1], [0], [1], [1], [1], [0], [0], [0], [0], [1], [0], [0], [1], [0], [0], [1], [0], [1], [0], [0]]

Classification Report:
Accuracy   | Precision  | Recall     | F-Score   
-----------------------------------
0.8500     | 0.8750     | 0.7778     | 0.8235    
